In [2]:
# Importing Libraries
import os
import json
import pandas as pd
import psycopg2
import streamlit as st
import plotly.express as px
from streamlit_option_menu import option_menu
from PIL import Image
import requests
import plotly.graph_objects as go

In [2]:
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        password = "changeme",
                        database = "phonepe_pulse_data",
                        port = "5432"
                        )
cursor = mydb.cursor()

In [5]:
#aggre_transaction

path1 = "D:/Program Files/Visual Studio/phonepe/pulse/data/aggregated/transaction/country/india/state/"
agg_tran_list = os.listdir(path1)

columns1 = {"States":[], "Years":[], "Quarter":[], "Transaction_type":[], "Transaction_count":[], "Transaction_amount":[]}

for state in agg_tran_list:
    cur_states = path1+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_year = cur_states+year+"/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year+file
            data = open(cur_file,"r")
            
            A = json.load(data)
            
            for i in A["data"]["transactionData"]:
                name = i["name"]
                count = i["paymentInstruments"][0]["count"]
                amount = i["paymentInstruments"][0]["amount"]
                columns1["Transaction_type"].append(name)
                columns1["Transaction_count"].append(count)
                columns1["Transaction_amount"].append(amount)
                columns1["States"].append(state)
                columns1["Years"].append(year)
                columns1["Quarter"].append(int(file.strip(".json")))
                
aggre_transaction = pd.DataFrame(columns1)

aggre_transaction["States"] = aggre_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
aggre_transaction["States"] = aggre_transaction["States"].str.replace("-"," ")
aggre_transaction["States"] = aggre_transaction["States"].str.title()
aggre_transaction['States'] = aggre_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")



In [ ]:
#aggregated transaction table
create_query1 = '''CREATE TABLE if not exists aggregated_transaction (States varchar(50),
                                                                      Years int,
                                                                      Quarter int,
                                                                      Transaction_type varchar(50),
                                                                      Transaction_count bigint,
                                                                      Transaction_amount bigint
                                                                      )'''
cursor.execute(create_query1)
mydb.commit()

for index,row in aggre_transaction.iterrows():
    insert_query1 = '''INSERT INTO aggregated_transaction (States, Years, Quarter, Transaction_type, Transaction_count, Transaction_amount)
                                                        values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Transaction_type"],
              row["Transaction_count"],
              row["Transaction_amount"]
              )
    cursor.execute(insert_query1,values)
    mydb.commit()

In [47]:
#aggre_user

path2 = "D:/Program Files/Visual Studio/phonepe/pulse/data/aggregated/user/country/india/state/"
agg_user_list = os.listdir(path2)

columns2 = {"States":[], "Years":[], "Quarter":[], "Brands":[], "Transaction_count":[], "Percentage":[]}

for state in agg_user_list:
    cur_states = path2+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_year = cur_states+year+"/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year+file
            data = open(cur_file,"r")
            
            B = json.load(data)
            
            try:
                for i in B["data"]["usersByDevice"]:
                    brand = i["brand"]
                    count = i["count"]
                    percentage = i["percentage"]
                    columns2["Brands"].append(brand)
                    columns2["Transaction_count"].append(count)
                    columns2["Percentage"].append(percentage)
                    columns2["States"].append(state)
                    columns2["Years"].append(year)
                    columns2["Quarter"].append(int(file.strip(".json")))
                    
            except:
                pass      
    
aggre_user = pd.DataFrame(columns2)

aggre_user["States"] = aggre_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
aggre_user["States"] = aggre_user["States"].str.replace("-"," ")
aggre_user["States"] = aggre_user["States"].str.title()
aggre_user['States'] = aggre_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


In [ ]:
#aggregated user table
create_query2 = '''CREATE TABLE if not exists aggregated_user (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                Brands varchar(50),
                                                                Transaction_count bigint,
                                                                Percentage float)'''
cursor.execute(create_query2)
mydb.commit()

for index,row in aggre_user.iterrows():
    insert_query2 = '''INSERT INTO aggregated_user (States, Years, Quarter, Brands, Transaction_count, Percentage)
                                                    values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Brands"],
              row["Transaction_count"],
              row["Percentage"])
    cursor.execute(insert_query2,values)
    mydb.commit()

In [73]:
#map_transaction

path3 = "D:/Program Files/Visual Studio/phonepe/pulse/data/map/transaction/hover/country/india/state/"
map_tran_list = os.listdir(path3)

columns3 = {"States":[], "Years":[], "Quarter":[], "District":[], "Transaction_count":[], "Transaction_amount":[]}

for state in map_tran_list:
    cur_states = path3+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_year = cur_states+year+"/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year+file
            data = open(cur_file,"r")
            
            C = json.load(data)
            
            for i in C["data"]["hoverDataList"]:
                    name = i["name"]
                    count = i["metric"][0]["count"]
                    amount = i["metric"][0]["amount"]
                    columns3["District"].append(name)
                    columns3["Transaction_count"].append(count)
                    columns3["Transaction_amount"].append(amount)
                    columns3["States"].append(state)
                    columns3["Years"].append(year)
                    columns3["Quarter"].append(int(file.strip(".json")))
                    
map_transaction = pd.DataFrame(columns3)

map_transaction["States"] = map_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
map_transaction["States"] = map_transaction["States"].str.replace("-"," ")
map_transaction["States"] = map_transaction["States"].str.title()
map_transaction['States'] = map_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


In [ ]:
#map_transaction_table
create_query3 = '''CREATE TABLE if not exists map_transaction (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                District varchar(50),
                                                                Transaction_count bigint,
                                                                Transaction_amount float)'''
cursor.execute(create_query3)
mydb.commit()

for index,row in map_transaction.iterrows():
            insert_query3 = '''
                INSERT INTO map_Transaction (States, Years, Quarter, District, Transaction_count, Transaction_amount)
                VALUES (%s, %s, %s, %s, %s, %s)

            '''
            values = (
                row['States'],
                row['Years'],
                row['Quarter'],
                row['District'],
                row['Transaction_count'],
                row['Transaction_amount']
            )
            cursor.execute(insert_query3,values)
            mydb.commit() 

In [51]:
#map_user

path4 = "D:/Program Files/Visual Studio/phonepe/pulse/data/map/user/hover/country/india/state/"
map_user_list = os.listdir(path4)

columns4 = {"States":[], "Years":[], "Quarter":[], "Districts":[], "RegisteredUser":[], "AppOpens":[]}

for state in map_user_list:
    cur_states = path4+state+"/"
    agg_year_list = os.listdir(cur_states)
    
    for year in agg_year_list:
        cur_year = cur_states+year+"/"
        agg_file_list = os.listdir(cur_year)
        
        for file in agg_file_list:
            cur_file = cur_year+file
            data = open(cur_file,"r")
            
            D = json.load(data)
            
            for i in D["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appopens = i[1]["appOpens"]
                columns4["Districts"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appopens)
                columns4["States"].append(state)
                columns4["Years"].append(year)
                columns4["Quarter"].append(int(file.strip(".json")))
                
map_user = pd.DataFrame(columns4)

map_user["States"] = map_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
map_user["States"] = map_user["States"].str.replace("-"," ")
map_user["States"] = map_user["States"].str.title()
map_user['States'] = map_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


In [ ]:
#map_user_table
create_query4 = '''CREATE TABLE if not exists map_user (States varchar(50),
                                                        Years int,
                                                        Quarter int,
                                                        Districts varchar(50),
                                                        RegisteredUser bigint,
                                                        AppOpens bigint)'''
cursor.execute(create_query4)
mydb.commit()

for index,row in map_user.iterrows():
    insert_query4 = '''INSERT INTO map_user (States, Years, Quarter, Districts, RegisteredUser, AppOpens)
                        values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Districts"],
              row["RegisteredUser"],
              row["AppOpens"])
    cursor.execute(insert_query4,values)
    mydb.commit()

In [53]:
#top_transaction

path5 = "D:/Program Files/Visual Studio/phonepe/pulse/data/top/transaction/country/india/state/"
top_tran_list = os.listdir(path5)

columns5 = {"States":[], "Years":[], "Quarter":[], "Pincodes":[], "Transaction_count":[], "Transaction_amount":[]}

for state in top_tran_list:
    cur_states = path5+state+"/"
    top_year_list = os.listdir(cur_states)
    
    for year in top_year_list:
        cur_years = cur_states+year+"/"
        top_file_list = os.listdir(cur_years)
        
        for file in top_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            E = json.load(data)

            for i in E["data"]["pincodes"]:
                entityName = i["entityName"]
                count = i["metric"]["count"]
                amount = i["metric"]["amount"]
                columns5["Pincodes"].append(entityName)
                columns5["Transaction_count"].append(count)
                columns5["Transaction_amount"].append(amount)
                columns5["States"].append(state)
                columns5["Years"].append(year)
                columns5["Quarter"].append(int(file.strip(".json")))
                
top_transaction = pd.DataFrame(columns5)

top_transaction["States"] = top_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_transaction["States"] = top_transaction["States"].str.replace("-"," ")
top_transaction["States"] = top_transaction["States"].str.title()
top_transaction['States'] = top_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")



In [ ]:
#top_transaction_table
create_query5 = '''CREATE TABLE if not exists top_transaction (States varchar(50),
                                                                Years int,
                                                                Quarter int,
                                                                pincodes int,
                                                                Transaction_count bigint,
                                                                Transaction_amount bigint)'''
cursor.execute(create_query5)
mydb.commit()

for index,row in top_transaction.iterrows():
    insert_query5 = '''INSERT INTO top_transaction (States, Years, Quarter, Pincodes, Transaction_count, Transaction_amount)
                                                    values(%s,%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Pincodes"],
              row["Transaction_count"],
              row["Transaction_amount"])
    cursor.execute(insert_query5,values)
    mydb.commit()

In [56]:
#top_user

path6 = "D:/Program Files/Visual Studio/phonepe/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path6)

columns6 = {"States":[], "Years":[], "Quarter":[], "Pincodes":[], "RegisteredUser":[]}

for state in top_user_list:
    cur_states = path6+state+"/"
    top_year_list = os.listdir(cur_states)

    for year in top_year_list:
        cur_years = cur_states+year+"/"
        top_file_list = os.listdir(cur_years)

        for file in top_file_list:
            cur_files = cur_years+file
            data = open(cur_files,"r")
            F = json.load(data)

            for i in F["data"]["pincodes"]:
                name = i["name"]
                registeredusers = i["registeredUsers"]
                columns6["Pincodes"].append(name)
                columns6["RegisteredUser"].append(registereduser)
                columns6["States"].append(state)
                columns6["Years"].append(year)
                columns6["Quarter"].append(int(file.strip(".json")))
                
top_user = pd.DataFrame(columns6)

top_user["States"] = top_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_user["States"] = top_user["States"].str.replace("-"," ")
top_user["States"] = top_user["States"].str.title()
top_user['States'] = top_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


In [ ]:
#top_user_table
create_query6 = '''CREATE TABLE if not exists top_user (States varchar(50),
                                                        Years int,
                                                        Quarter int,
                                                        Pincodes int,
                                                        RegisteredUser bigint
                                                        )'''
cursor.execute(create_query6)
mydb.commit()

for index,row in top_user.iterrows():
    insert_query6 = '''INSERT INTO top_user (States, Years, Quarter, Pincodes, RegisteredUser)
                                            values(%s,%s,%s,%s,%s)'''
    values = (row["States"],
              row["Years"],
              row["Quarter"],
              row["Pincodes"],
              row["RegisteredUser"])
    cursor.execute(insert_query6,values)
    mydb.commit()

In [3]:
# Fetch GeoJSON data
url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response = requests.get(url)
data1 = json.loads(response.content)

# Extract state names and sort them in alphabetical order
state_names = [feature['properties']['ST_NM'] for feature in data1['features']]
state_names.sort()

In [ ]:
state_names

In [ ]:
# Creating connection with MySQL workbench and retrieve the data from the "agg_trans" table as dataframe:
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        password = "changeme",
                        database = "phonepe_pulse",
                        port = "5432"
                        )
cursor = mydb.cursor()

cursor.execute("SELECT states, years, quarter, transaction_type, transaction_count, transaction_amount FROM aggregated_transaction")
data = cursor.fetchall()
columns = ["state", "year", "quarter", "transaction_type", "transaction_count", "transaction_amount"]
df = pd.DataFrame(data, columns=columns)
df

#Find unique values in column 'A'
pd.DataFrame(df['state'].unique())

In [20]:
# Assuming your DataFrame 'df' looks like this
# df = pd.DataFrame({'states': [...state_names...]})

# Define the conversion function
def convert_state_name(state):
    return state.replace('-', ' ').title()

# Apply the conversion function to the 'states' column in the DataFrame
df['state'] = df['state'].apply(lambda x: convert_state_name(x))

# Assuming you have already converted and updated the DataFrame 'df'
df.to_csv('D:/Program Files/Visual Studio/phonepe/data/india_map.csv', index=False)

In [ ]:
# Read the "india_map" CSV file into a DataFrame
df1 = pd.read_csv('D:/Program Files/Visual Studio/phonepe/data/india_map.csv')
df1.head(4)

In [22]:
import pandas as pd

# Assuming df is your DataFrame
min_transaction_amount = df['transaction_amount'].min()
max_transaction_amount = df['transaction_amount'].max()

print("Minimum Transaction Amount:", min_transaction_amount)
print("Maximum Transaction Amount:", max_transaction_amount)

Minimum Transaction Amount: 34
Maximum Transaction Amount: 2393380395297


In [23]:
# Geo plot


fig_tra = px.choropleth(
    df1,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='state',
    color='transaction_amount',
    color_continuous_scale='thermal',
    range_color=(df1['transaction_amount'].min(), df1['transaction_amount'].max()),  # Set custom range
    title='Transaction Analysis'
)

fig_tra.update_geos(fitbounds="locations", visible=False)
fig_tra.update_layout(title_font=dict(size=33), title_font_color='#6739b7', height=800)

st.plotly_chart(fig_tra, use_container_width=True)

2024-01-22 00:14:05.538 
  command:

    streamlit run C:\Users\sggur\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [ ]:
import pandas as pd
import plotly.express as px
import streamlit as st

# Read the "india_map" CSV file into a DataFrame
# Assuming df1 is your DataFrame containing the data
df1 = pd.read_csv(r'D:/Program Files/Visual Studio/phonepe/data/india_map.csv')
df1.head(4)

# Debugging statements
print("Data Types:")
print(df1.dtypes)

print("Null Values:")
print(df1.isnull().sum())

print("Transaction Amount Summary:")
print(df1['transaction_amount'].describe())

# Calculate the range of transaction_amount for setting zmin and zmax
zmin = df1['transaction_amount'].min()
zmax = df1['transaction_amount'].max()

fig_tra = px.choropleth(
    df1,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='state',
    color='transaction_amount',
    color_continuous_scale='Viridis',  # Change the color scale
    range_color=(zmin, zmax),  # Set custom range
    title='Transaction Analysis'
)

fig_tra.update_geos(fitbounds="locations", visible=False)
fig_tra.update_layout(title_font=dict(size=33), title_font_color='#6739b7', height=800)

st.plotly_chart(fig_tra, use_container_width=True)

In [ ]:
# data query for map_trans :
# Creating connection with postgressql and retrieve the data from the "agg_trans" table as dataframe:
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        password = "changeme",
                        database = "phonepe_pulse",
                        port = "5432"
                        )
cursor = mydb.cursor()

cursor.execute("SELECT states, years, quarter, District, transaction_count, transaction_Amount FROM map_transaction")
data = cursor.fetchall()
columns = ["state", "year", "quarter", "District", "Count", "Amount"]
df3 = pd.DataFrame(data, columns=columns)
df3

In [27]:
# download to local drive as csv file ;
df3.to_csv('D:/Program Files/Visual Studio/phonepe/data/histogram_district.csv', index=False)

In [ ]:
# for voline plot :
cursor.execute("SELECT States, Years, Quarter, Districts, Registereduser, Appopens FROM map_user")
data = cursor.fetchall()
columns = ["State", "Year", "Quarter", "District", "RegisteredUser", "AppOpens"]
df4 = pd.DataFrame(data, columns=columns)
df4

In [29]:
# download to local drive as csv file ;
df4.to_csv('D:/Program Files/Visual Studio/phonepe/data/violin_plot.csv', index=False)

In [30]:
# Assuming your DataFrame 'df' looks like this
# df = pd.DataFrame({'states': [...state_names...]})

# Define the conversion function
def convert_state_name(state):
    return state.replace('-', ' ').title()

# Apply the conversion function to the 'states' column in the DataFrame
df4['State'] = df4['State'].apply(lambda x: convert_state_name(x))

# Assuming you have already converted and updated the DataFrame 'df'
df4.to_csv('D:/Program Files/Visual Studio/phonepe/data/state_map.csv', index=False)

In [ ]:
# read the "state_map" CSV file into a DataFrame:
df4 = pd.read_csv('D:/Program Files/Visual Studio/phonepe/data/state_map.csv')
df4

In [ ]:
cursor = mydb.cursor()

cursor.execute("SELECT states, years, quarter, transaction_type, transaction_count, transaction_amount FROM aggregated_transaction")
data = cursor.fetchall()
columns = ["state", "year", "quarter", "transaction_type", "transaction_count", "transaction_amount"]
df = pd.DataFrame(data, columns=columns)
df

In [ ]:
df['transaction_amount'].describe()

In [ ]:
# data query for map_trans :
# Creating connection with postgressql and retrieve the data from the "aggregated_user" table as dataframe:
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        password = "changeme",
                        database = "phonepe_pulse",
                        port = "5432"
                        )
cursor = mydb.cursor()

cursor.execute("SELECT states, years, quarter, Brands, transaction_count, percentage FROM aggregated_user")
data = cursor.fetchall()
columns = ["state", "year", "quarter", "Brands", "Count", "Percentage"]
df5 = pd.DataFrame(data, columns=columns)
df5

In [5]:
# download to local drive as csv file ;
df5.to_csv('D:/Program Files/Visual Studio/New folder/data/bar_chart.csv', index=False)